# Assignment 1

## 1: Load and Inspect the Data

Load the datasets (employee_rating.csv and employee_general.csv) and check if they are loaded properly. Print data types and classify them as numeric, binary, ordinal, or nominal.

In [27]:
# Importing neccessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [28]:
# Load the datasets
rating_df = pd.read_csv('employee_rating.csv')
general_df = pd.read_csv('employee_general.csv', delimiter='|')  # Use '|' as delimiter since columns are concatenated


In [29]:
# Display first few rows
print("Employee Rating Data:\n", rating_df.head(), "\n")
print("Employee General Data:\n", general_df.head(), "\n")


Employee Rating Data:
    EmployeeNumber              Department EnvironmentSatisfaction  \
0               1                   Sales                  Medium   
1               2  Research & Development                    High   
2               4  Research & Development               Very High   
3               5  Research & Development               Very High   
4               7  Research & Development                     Low   

                 JobRole PerformanceRating  YearsAtCompany  \
0        Sales Executive         Excellent               6   
1     Research Scientist       Outstanding              10   
2  Laboratory Technician         Excellent               0   
3     Research Scientist         Excellent               8   
4  Laboratory Technician         Excellent               2   

   YearsInCurrentRole  MonthlyIncome  
0                   4           5993  
1                   7           5130  
2                   0           2090  
3                   7           2

In [30]:
# Check data types
print("Data Types in Employee Rating Dataset:\n", rating_df.dtypes, "\n")
print("Data Types in Employee General Dataset:\n", general_df.dtypes, "\n")


Data Types in Employee Rating Dataset:
 EmployeeNumber              int64
Department                 object
EnvironmentSatisfaction    object
JobRole                    object
PerformanceRating          object
YearsAtCompany              int64
YearsInCurrentRole          int64
MonthlyIncome               int64
dtype: object 

Data Types in Employee General Dataset:
 EmployeeNumber     int64
Education         object
EducationField    object
MaritalStatus     object
dtype: object 



In [31]:
# Identify column types (numeric, binary, ordinal, nominal)
def identify_column_types(df):
    """
    Identifies the data type of each column in a Pandas DataFrame.

    Args:
        df: The input Pandas DataFrame.

    Returns:
        A dictionary where keys are column names and values are
        the identified data types ('numeric', 'binary', 'ordinal', 'nominal').
    """
    column_types = {}
    for column in df.columns:
        unique_values = df[column].unique()
        if pd.api.types.is_numeric_dtype(df[column]):
            if len(unique_values) == 2 and all(x in [0, 1] or pd.isna(x) for x in unique_values):
                column_types[column] = 'binary'  # Example: 0/1 values
            else:
                column_types[column] = 'numeric'
        elif pd.api.types.is_categorical_dtype(df[column]):
             column_types[column] = 'nominal'
        elif all(isinstance(val, str) for val in unique_values):
            column_types[column] = 'nominal' #treat string columns as nominal by default
        else:
          column_types[column] = 'unknown' #add unknown category to handle ambiguous types

    return column_types

# Analyze column types for both DataFrames
rating_column_types = identify_column_types(rating_df)
general_column_types = identify_column_types(general_df)

print("Column Types in Employee Rating Dataset:\n", rating_column_types, "\n")
print("Column Types in Employee General Dataset:\n", general_column_types, "\n")

Column Types in Employee Rating Dataset:
 {'EmployeeNumber': 'numeric', 'Department': 'nominal', 'EnvironmentSatisfaction': 'nominal', 'JobRole': 'nominal', 'PerformanceRating': 'nominal', 'YearsAtCompany': 'numeric', 'YearsInCurrentRole': 'numeric', 'MonthlyIncome': 'numeric'} 

Column Types in Employee General Dataset:
 {'EmployeeNumber': 'numeric', 'Education': 'nominal', 'EducationField': 'nominal', 'MaritalStatus': 'nominal'} 



In [32]:
# Identify column types (numeric, binary, ordinal, nominal)
print("Column Types in Employee Rating Dataset:\n", rating_df.nunique(), "\n")
print("Column Types in Employee General Dataset:\n", general_df.nunique(), "\n")

Column Types in Employee Rating Dataset:
 EmployeeNumber             1470
Department                    3
EnvironmentSatisfaction       4
JobRole                       9
PerformanceRating             2
YearsAtCompany               37
YearsInCurrentRole           19
MonthlyIncome              1349
dtype: int64 

Column Types in Employee General Dataset:
 EmployeeNumber    1470
Education            5
EducationField       6
MaritalStatus        3
dtype: int64 



## 2: Log Transformation of MonthlyIncome

Transform the MonthlyIncome column by applying a log transformation to reduce skewness.

In [33]:
# Apply log transformation to MonthlyIncome
rating_df['log_income'] = np.log(rating_df['MonthlyIncome'])


In [34]:
# Check skewness
print("Skewness after log transformation:", rating_df['log_income'].skew())

Skewness after log transformation: 0.2861922926389174


## 3: Convert Categorical Variables in employee_rating

Identify categorical columns in employee_rating that can be converted to numerical values and perform the transformation.



In [35]:
# Identify unique values in categorical columns
for col in rating_df.select_dtypes(include=['object']).columns:
    print(f"Unique values in {col}: {rating_df[col].unique()}")

Unique values in Department: ['Sales' 'Research & Development' 'Human Resources']
Unique values in EnvironmentSatisfaction: ['Medium' 'High' 'Very High' 'Low']
Unique values in JobRole: ['Sales Executive' 'Research Scientist' 'Laboratory Technician'
 'Manufacturing Director' 'Healthcare Representative' 'Manager'
 'Sales Representative' 'Research Director' 'Human Resources']
Unique values in PerformanceRating: ['Excellent' 'Outstanding']


In [36]:
# EnvironmentSatisfaction: Ordinal (1:Low, 2:Medium, 3:High, 4:Very High)
rating_df['EnvironmentSatisfaction'] = rating_df['EnvironmentSatisfaction'].map({'Low': 1, 'Medium': 2, 'High': 3, 'Very High': 4})


In [37]:
# Display data after transformation
print(rating_df.head())


   EmployeeNumber              Department  EnvironmentSatisfaction  \
0               1                   Sales                        2   
1               2  Research & Development                        3   
2               4  Research & Development                        4   
3               5  Research & Development                        4   
4               7  Research & Development                        1   

                 JobRole PerformanceRating  YearsAtCompany  \
0        Sales Executive         Excellent               6   
1     Research Scientist       Outstanding              10   
2  Laboratory Technician         Excellent               0   
3     Research Scientist         Excellent               8   
4  Laboratory Technician         Excellent               2   

   YearsInCurrentRole  MonthlyIncome  log_income  
0                   4           5993    8.698347  
1                   7           5130    8.542861  
2                   0           2090    7.644919  
3 

## 4: Convert categorical (class) columns in `general_df` to meaningful numbers as transformation


In [41]:
# Convert categorical columns in `general_df` to meaningful numbers

# Education (Ordinal Mapping)
education_mapping = {'Below College': 1, 'College': 2, 'Bachelor': 3, 'Master': 4}
general_df['Education'] = general_df['Education'].map(education_mapping)

# Marital Status (Nominal Mapping)
marital_mapping = {'Single': 1, 'Married': 2, 'Divorced': 3}
general_df['MaritalStatus'] = general_df['MaritalStatus'].map(marital_mapping)

# EducationField (Nominal - One-Hot Encoding)
general_df = pd.get_dummies(general_df, columns=['EducationField'], drop_first=True)


In [42]:
# Display data after transformation
print(general_df.head())

   EmployeeNumber  Education  MaritalStatus  EducationField_Life Sciences  \
0               1        2.0              1                          True   
1               2        1.0              2                          True   
2               4        2.0              1                         False   
3               5        4.0              2                          True   
4               7        1.0              2                         False   

   EducationField_Marketing  EducationField_Medical  EducationField_Other  \
0                     False                   False                 False   
1                     False                   False                 False   
2                     False                   False                  True   
3                     False                   False                 False   
4                     False                    True                 False   

   EducationField_Technical Degree  
0                            False  


## 5. Merge both datasets properly

In [45]:
# Step 2: Merge datasets on EmployeeNumber
merged_df = pd.merge(rating_df, general_df, on='EmployeeNumber')

# Display the first few rows of the merged dataset
print("\nMerged Dataset:\n", merged_df.head())



Merged Dataset:
    EmployeeNumber              Department  EnvironmentSatisfaction  \
0               1                   Sales                        2   
1               2  Research & Development                        3   
2               4  Research & Development                        4   
3               5  Research & Development                        4   
4               7  Research & Development                        1   

                 JobRole PerformanceRating  YearsAtCompany  \
0        Sales Executive         Excellent               6   
1     Research Scientist       Outstanding              10   
2  Laboratory Technician         Excellent               0   
3     Research Scientist         Excellent               8   
4  Laboratory Technician         Excellent               2   

   YearsInCurrentRole  MonthlyIncome  log_income  Education  MaritalStatus  \
0                   4           5993    8.698347        2.0              1   
1                   7           

In [46]:
# Check final data types
print("\nFinal Data Types:\n", merged_df.dtypes)


Final Data Types:
 EmployeeNumber                       int64
Department                          object
EnvironmentSatisfaction              int64
JobRole                             object
PerformanceRating                   object
YearsAtCompany                       int64
YearsInCurrentRole                   int64
MonthlyIncome                        int64
log_income                         float64
Education                          float64
MaritalStatus                        int64
EducationField_Life Sciences          bool
EducationField_Marketing              bool
EducationField_Medical                bool
EducationField_Other                  bool
EducationField_Technical Degree       bool
dtype: object
